In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/2021-ai-w4-p1/submit.csv
/kaggle/input/2021-ai-w4-p1/train.csv
/kaggle/input/2021-ai-w4-p1/test.csv


In [2]:
import torch
import torch.optim as optim

In [3]:
torch.manual_seed(1)

In [4]:
train_data=pd.read_csv("/kaggle/input/2021-ai-w4-p1/train.csv")
test_data=pd.read_csv("/kaggle/input/2021-ai-w4-p1/test.csv")
submit_data=pd.read_csv("/kaggle/input/2021-ai-w4-p1/submit.csv")
print(train_data.head())
print(test_data.head())
print(submit_data.head())

   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            1       95             60             18       58  23.9   
1            5      105             72             29      325  36.9   
2            0      135             68             42      250  42.3   
3            4      131             68             21      166  33.1   
4            1      103             30             38       83  43.3   

   DiabetesPedigreeFunction  Age  Diabetes  
0                     0.260   22         0  
1                     0.159   28         0  
2                     0.365   24         1  
3                     0.160   28         0  
4                     0.183   33         0  
   Pregnancies  Glucose  BloodPressure  SkinThickness  Insulin   BMI  \
0            6       98             58             33      190  34.0   
1            2      112             75             32        0  35.7   
2            2      108             64              0        0  30.8   
3        

In [5]:
#0으로 빠진데이터 체크
print((train_data.iloc[:,5]==0).sum())
print((test_data.iloc[:,5]==0).sum())

5
6


In [6]:
#평균값으로 체우기
idx = (train_data.iloc[:, 1] == 0)
gluMean = train_data['Glucose'][~idx].mean()
train_data.iloc[idx, 1] = gluMean
idx = (train_data.iloc[:, 2] == 0)
BPMean = train_data['BloodPressure'][~idx].mean()
train_data.iloc[idx, 2] = BPMean
idx = (train_data.iloc[:, 5] == 0)
mbiMean = train_data['BMI'][~idx].mean()
train_data.iloc[idx, 5] = mbiMean
print(gluMean, BPMean, mbiMean)

121.98496240601504 72.24131274131274 32.27612781954887


In [7]:
#확인용
print((train_data.iloc[:,5]==0).sum())
print((test_data.iloc[:,5]==0).sum())

0
6


In [8]:
xtrain=np.array([train_data['Pregnancies'],train_data['Glucose'],train_data['BloodPressure'],train_data['SkinThickness'],train_data['Insulin'],train_data['BMI'],train_data['DiabetesPedigreeFunction'],train_data['Age']])
xtrain=torch.FloatTensor(xtrain.T)
ytrain=torch.FloatTensor(train_data['Diabetes'])
print(xtrain.shape)
print(ytrain.shape)
print(ytrain.unsqueeze(1).shape)

torch.Size([537, 8])
torch.Size([537])
torch.Size([537, 1])


In [9]:
W=torch.zeros((8,1), requires_grad=True)
b=torch.zeros(1, requires_grad=True)

loss = torch.nn.BCELoss()
optimizer = optim.SGD([W,b], lr=0.0002)

nb_epochs=500000
for epoch in range(nb_epochs + 1):
    hypothesis=torch.sigmoid(xtrain.matmul(W)+b)
    #cost = -(ytrain*torch.log(hypothesis)+(1-ytrain)*torch.log(1-hypothesis)).mean()
    cost = loss(hypothesis, ytrain.unsqueeze(1))
    optimizer.zero_grad()
    cost.backward()
    optimizer.step()
    
    if epoch%50000==0:
        print('Epoch {:4d}/{} Cost: {:.6f}'.format(epoch, nb_epochs, cost.item()))

Epoch    0/500000 Cost: 0.693147
Epoch 50000/500000 Cost: 0.570411
Epoch 100000/500000 Cost: 0.562370
Epoch 150000/500000 Cost: 0.554915
Epoch 200000/500000 Cost: 0.548001
Epoch 250000/500000 Cost: 0.541583
Epoch 300000/500000 Cost: 0.535634
Epoch 350000/500000 Cost: 0.530100
Epoch 400000/500000 Cost: 0.524944
Epoch 450000/500000 Cost: 0.520195
Epoch 500000/500000 Cost: 0.515733


In [10]:
print(W)
print(b)

tensor([[ 0.0674],
        [ 0.0254],
        [-0.0477],
        [-0.0072],
        [-0.0003],
        [ 0.0469],
        [ 0.1494],
        [ 0.0173]], requires_grad=True)
tensor([-2.4984], requires_grad=True)


In [11]:
#test데이터에도 0값 채우기
idx = (test_data.iloc[:, 1] == 0)
test_data.iloc[idx, 1] = gluMean
idx = (test_data.iloc[:, 2] == 0)
test_data.iloc[idx, 2] = BPMean
idx = (test_data.iloc[:, 5] == 0)
test_data.iloc[idx, 5] = mbiMean

In [12]:
print((train_data.iloc[:,5]==0).sum())
print((test_data.iloc[:,5]==0).sum())

0
0


In [13]:
xtest=torch.FloatTensor(np.array([test_data['Pregnancies'],test_data['Glucose'],test_data['BloodPressure'],test_data['SkinThickness'],test_data['Insulin'],test_data['BMI'],test_data['DiabetesPedigreeFunction'],test_data['Age']]))
xtest=xtest.T
ytest=torch.sigmoid(xtest.matmul(W)+b)
idx = ytest >= torch.FloatTensor([0.5])
ytest[idx]=1
ytest[~idx]=0
print(ytest[1:5])

tensor([[0.],
        [0.],
        [0.],
        [0.]], grad_fn=<SliceBackward>)


In [14]:
submit_data['Diabetes']=ytest.detach().int().numpy()
print(submit_data.head())

   ID  Diabetes
0   0         0
1   1         0
2   2         0
3   3         0
4   4         0


In [15]:
submit_data.to_csv("submission.csv",mode='w',index=False)